# Regresión logística

## Índice
1. **[Modelo de regresión logística](#1.La-regresión-logística)**
    * **[1.1 ¿Qué es la regresión logística?](#1.1-¿Qué-es-la-regresión-logística?)**
    * **[1.2 Implementación de la regresión logística](#1.2-Implementación-de-la-regresión-logística)**
    * **[1.3 Entrenamiento](#1.3-Entrenamiento)**
        - **[1.3.1 Dataset de ejemplo](#1.3.1-Dataset-de-ejemplo)**
        - **[1.3.2 Visualización del entrenamiento](#1.3.2-Visualización-del-entrenamiento)**
2. **[Regresión logística con scikit-learn](#2.-Regresión-logística-con-scikit-learn)**
    * **[2.1 Interfaz del modelo](#2.1-Interfaz-del-modelo)**
    * **[2.2 Visualización de resultados](#2.2-Visualización-de-resultados)**
    
3. **[Evaluación del modelo](#3.-Evaluación-del-modelo)**
    * **[3.1 Matriz de confusión](#3.1-Matriz-de-confusión)**
    * **[3.2 Métricas de clasificación](#3.2-Métricas-de-clasificación)**
    
4. **[Clasificación multiclase](#4.-Clasificación-multiclase)**
    * **[4.1 Fronteras de decisión](#4.1-Fronteras-de-decisión)**
    * **[4.2 Evaluación del modelo](#4.2-Evaluación-del-modelo)**
    
     
 


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris, make_blobs
from sklearn.metrics import (confusion_matrix, accuracy_score,
                             precision_score, recall_score,
                             roc_curve, roc_auc_score, classification_report)

from mglearn.datasets import make_forge
from mglearn.plots import plot_2d_separator, plot_2d_classification
from mglearn import discrete_scatter

from code_utils import plot_dataset_2d, plot_model_output, RegLogTrainingPlotter
                      
import holoviews as hv
import panel as pn
hv.extension("bokeh")
pn.extension()

## [1.](#Índice)  Modelo de regresión logística

### [1.1](#Índice) ¿Qué es la regresión logística?

Para hacer clasificación lineal podemos usar algoritmos diferentes a los vistos anteriormente. Entre estos algoritmos encontramos la regresión logística. A pesar del nombre, **NO sirve para hacer regresión y solo se usa para clasificación**.

![linear to logistic](./imgs/LR_model_diagram.png)

Normalmente se usa para hacer clasificación binaria, aunque se puede extender (la regresión logística y otros algoritmos de clasificación binaria) a clasificación multiclase como luego veremos.

Este caso es similar a lo que vimos anteriormente, tenemos lo siguiente:

$$ \hat{y}(x) = w_0 · x_0 + w_1 · x_1 + ... + w_n · x_n $$

Salvo que en este caso sumaremos una constante al output de nuestro modelo lineal de la siguiente forma:

$$ P(y=1|x) = b + w_0 · x_0 + w_1 · x_1 + ... + w_n · x_n $$

Donde **$x_i$ representa cada una de las features** de nuestro ejemplo, y **$w_i$ representa un vector de pesos**. Por eso, además de encontrar un peso para cada una de las features, nuestro modelo tendrá un parámetro adicional que corresponderá a sumar una constante.

Esta constante sumada al output de un modelo lineal se llama **intercept**.



Y nuestras clases las podemos definir como:

$$ clase_{1} = 1 \space si \space P(y=1|x)\ge 0.5 $$
$$ clase_{0} = 0 \space si \space P(y=1|x)\lt 0.5 $$

<div class="alert alert-warning">
Las clases da igual como las definamos, pueden ser "Sí" y "No" o "No" y "Sí", es decir, simplemente tiene que ser coherente con la pregunta.

¿Es azul? Sí -> 0 / No -> 1

¿Es azul? Sí -> 1 / No -> 0
</div>

Lo que queremos son probabilidades que nos digan si algo es más probable que esté en una clase o en otra. Para ello se usa la función logística o sigmoide (por su forma en ese):

$$ S(x_i) = \frac{1}{1+e^{-x_i}} $$


![linear to logistic](./imgs/13_linear_vs_logistic_regression.webp)

## [1.2](#Índice) Implementación de la regresión logística

Con toda esta chicha vamos a construir nuestro algoritmo de regresión logística y evaluar como funciona sobre los datos creados:

In [ ]:
class RegLog:
    """
    Modelo de regresión logistica.
    
    Este modelo solo puede usarse para clasificación binaria (clases {1, 0}). \
     Una vez entrenado, modela la probabilidad de que un ejemplo pertenezca a la clase 1. 
     
    Puede ser usado para predecir la probabilidad de pertenecer a la clase uno, o para
     predecir directamente la clase asignada a cada ejemplo.
    """
    def __init__(self, learning_rate: float = 0.001, num_iters: int = 50_000):
        """
        Inicializa una instancia de Reglog. 
        
        Args:
            learning_rate: Este valor define cuanto va a actualizarse el model con su \
                           correspondiente gradiente en cada iteración de entrenamiento.
            num_iters: Número de iteraciones  que se van a llevar a cabo para \
                        entrenar el modelo.
        """
        self.learning_rate = learning_rate
        self.num_iters = num_iters
        self.weights = None # Será un vector de longitud igual al numero de features + 1.
    
    def __repr__(self) -> str:
        string = "{}\n".format(self.__class__.__name__)
        if self.weights is None:
            string += "Modelo no entrenado."
        else:
            w_columns = ["weight_{}: {:.4f} ".format(i, w) for i, w in enumerate(self.weights[1:])]
            string += "".join(w_columns)
            string += "intercept: {:.4f}".format(self.weights[0])
        return string
        
    @staticmethod
    def sigmoid(linear_output: np.ndarray) -> np.ndarray:
        """
        Aplica la función sigmoide a cada uno de los valores del array de entrada.

        Args:
            linear_output: Corresponde a la predicción de un modelo lineal.

        Returns:
            Aray que contiene el resultado de aplicar la función sigmoide al \
            valor de entrada.
        """
        return 1 / (1 + np.exp(-linear_output))

    @staticmethod
    def log_likelihood(sigmoid_output, y: np.ndarray) -> np.ndarray:
        """
        Función de log-likehood que puede usarse para cuantificar el error del modelo.

        Args:
            sigmoid_output: Predicción del modelo en forma de probabilidad.
            y: Valor real correspondiente a cada predicción realizada por el modelo.

        Returns:
            Logaritmo de la función de verosimilitud correspondiente a los valores de entrada.
        """
        return (-y * np.log(sigmoid_output) - (1 - y) * np.log(1 - sigmoid_output)).mean()
    
    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        """
        Para cada ejemplo, predice la probabilidad de pertenecer a la clase 1.

        Args:
            X: Dataset que se desea clasificar. Cada ejemplo corresponde a una \
               fila (dimensión 0 del array), y cada feature a una columna \
               (dimensión 1 del array).

        Returns:
            Array que contiene las probabilidades assignadas a cada ejemplo de pertenecer a la
            clase 1. Este vector contiene floats en el intervalo [0, 1].
        """
        # El efecto del intercept es sumar una constante a la predicción lineal del modelo.
        # Al aplicar el producto escalar el peso asignado al intercept se multiplica por 1, 
        # lo cual equivale a sumar el peso, pero es más eficiente de calcular.
        intercept = np.ones((X.shape[0], 1))
        x = np.concatenate((intercept, X), axis=1)
        return self.sigmoid(np.dot(x, self.weights))

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Predice la classe a la que pertenecen cada uno de los ejemplos de X.

        Args:
            X: Dataset que se desea clasificar. Cada ejemplo corresponde a una \
               fila (dimensión 0 del array), y cada feature a una columna \
               (dimensión 1 del array).

        Returns:
            Devuelve un array unidmiensional que contiene las predicciones para cada ejemplo de X.
            Este vector solo puede contener los valores {0, 1}
        """
        return self.predict_proba(X).round()

    def fit(self, X_train: np.ndarray, y_train: np.ndarray) -> None:
        """
        Entrena el modelo de regresión logística.

        Args:
            X_train: Dataset de entrenamiento. Este array contiene \
               las features como columnas y los ejemplos de \
               entrenamiento como filas.

            y_train: Classes a las que pertenecen los ejemplos \
               del dataset de entrenamiento codificadas como \
               1 o 0. Cada elemento de este vector corresponde \
               al ejemplo x con el mismo índice. (Un elemento por \
               cada fila de x)
        """        
        self._initialize_weights(X_train)
        for i in range(self.num_iters):
            self.training_iteration(X_train, y_train)

    def training_iteration(self, X_train: np.ndarray, y_train: np.ndarray) -> None:
        """
        Actualiza los pesos del modelo realizando una iteración del algoritmo gradient descend.
        """
        prediction = self.predict_proba(X_train)
        loss = self.log_likelihood(prediction, y_train)
        # Guardar estos valores es un truco para calcular el gradiente mas eficientemente.
        self._prediction, self._y_train, self._X_train = prediction, y_train, X_train
        # En general, el gradiente se calcula derivando la función 
        # de pérdida usando la regla de la cadena.
        loglike_gradient = self._calculate_gradient(loss)
        self.weights -= self.learning_rate * loglike_gradient
    
    def _initialize_weights(self, X: np.ndarray) -> None:
        """Inicializa los pesos del modelo."""
        n_features = X.shape[1]
        self.weights = np.zeros(n_features + 1) # El + 1 corresponde al peso de intercept.        

    def _calculate_gradient(self, loss: np.ndarray) -> np.ndarray:
        """
        Devuelve el gradiente de la función log likelihood.

        Args:
            loss: Función de coste que va a derivarse. Será ignorado porque utilizaremos un truco \
                  matemático que no requiere del valor de la función de loss, y que es mas 
                  eficiente de calcular.

        Returns:
            Array que contiene el gradiente de loss.
        """
        # Con un poco de matemáticas, podemos darnos cuenta de que el gradiente del 
        # log-likelihood no es más que la multiplicación de la matriz de entradas transpuesta (X.T)
        # por el error de la predicción.
        intercept = np.ones((self._X_train.shape[0], 1))
        X = np.concatenate((intercept, self._X_train), axis=1)
        gradient = np.dot(X.T, (self._prediction - self._y_train)) / self._y_train.size
        return gradient


### [1.3](#Índice) Entrenamiento

A continuación mostraremos como entrenar nuestro modelo de regresión logística.

#### [1.3.1](#Índice) Dataset de ejemplo

Para ello utilizaremos un dataset de ejemplo con dos features diferentes y dos clases. Al tener solo dos features, vamos a poder visualizar fácilmente como nuestro modelo classifica los diferentes ejemplos.

In [ ]:
X_forge, y_forge = make_forge()
X_train, X_test, y_train, y_test = train_test_split(X_forge, y_forge, random_state=0)

In [ ]:
plot_dataset_2d(X_train, y_train, X_test, y_test).opts(xlim=(7.5, 12.3), ylim=(-1.5, 6))

#### [1.3.2](#Índice) Visualización del entrenamiento

Instanciamos nuestra clase recien creada, y los gráficos en los que visualizaremos como evolucionan la función de pérdida y los pesos del modelo durante el proceso de entrenamiento.

In [ ]:
reglog = RegLog(num_iters=5500, learning_rate=0.001)
reglog

In [ ]:
plot = RegLogTrainingPlotter(reglog, plot_every=150)
plot.create_plot()

Para ajustar el modelo a los datos llamaremos al método `fit()`.

In [ ]:
plot.fit(X_train, y_train, X_test, y_test)

Podéis modificar la tasa de aprendizaje o el número de iteraciones para ver si mejora o empeora.

Predecimos con nuestro modelo ajustado los valores de prueba para ver si se parecen a las etiquetas:

In [ ]:
y_pred = reglog.predict(X_test)
reglog

In [ ]:
for i, (yi, yyi, x0, x1) in enumerate(zip(y_pred, y_test, X_test[:,0], X_test[:,1])):
    print("Example {}, prediction {} true value {} feature_0 {:.3f} feature_1 {:.3f}".format(i, int(yi), yyi, x0, x1))

## [2.](#Índice) Regresión logística con scikit-learn

La libreria scikit-learn ofrece un modelo de regresión logística con más funcionalidades que el mostrado anteriormente, y su uso es muy similar al de la regresión lineal vista anteriormente.

### [2.1](#Índice) Interfaz del modelo

Vamos a comparar con lo que trae `scikit-learn`. Instanciamos:

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg

Varias cosas de las que se ven ahí arriba. 

* El *solver* es lo que realiza la optimización. Podéis leer más sobre ello aquí: https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html. En el caso de scikit-learn usa *solvers* especializados. Más info aquí: https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

* `C` es el parámetro que regula la regularización. Un valor bajo regulariza mucho mientras que un valor muy alto le quita importancia a la regularización.

* `class_weight` nos permite dar más peso a alguna clase (acordáos de los datos desnivelados o descompensados (*imbalanced datasets*).

* `penalty` define el tipo de regularización. Dependiendo del *solver* se podrán usar unos tipos de regularizaciones u otros.

* ...

Si queremos que el modelo de sklearn se comporte como nuestro modelo anterior tenemos que instanciarlo con los siguientes parámetros.

In [ ]:
logreg = LogisticRegression(C=1e20, fit_intercept=False)

Ajustamos:

In [ ]:
logreg.fit(X_train, y_train)

Predecimos:

In [ ]:
y_pred = logreg.predict(X_test)

### [2.2](#Índice) Visualización de resultados

A continuación comprobaremos como se comporta el modelo entrenado:

In [ ]:
for i, (yi, yyi) in enumerate(zip(y_pred, y_test)):
    print("Example {}, prediction {} true value {}".format(i, int(yi), yyi))

Parece que lo que obtenemos es similar a nuestro algoritmo de más arriba. Vamos a ver los pesos que obtenemos:

In [ ]:
print(logreg.intercept_, logreg.coef_)

In [ ]:
default_logreg = LogisticRegression().fit(X_train, y_train)
plot_model_output(default_logreg, X_train, y_train).opts(title="Clasificacion del dataset de entrenamiento")

In [ ]:
plot_model_output(default_logreg, X_test, y_test).opts(title="Clasificacion del dataset de test")

## [3.](#Índice) Evaluación del modelo

In [ ]:
from code_utils import (plot_confussion_matrix, plot_decision_boundaries, plot_decision_boundaries,
                        interactive_logistic_regression, plot_model_evaluation, plot_feature_importances,
                        plot_classification_report
                       )

### [3.1](#Índice) Matriz de confusión

Ya hemos hablado de la matriz de confusión. Vamos a ver como se ven los modelos usando la matriz de confusión:

In [ ]:
logreg = LogisticRegression(C=0.1).fit(X_train, y_train)

y_pred = logreg.predict(X_test)

conf_mat = confusion_matrix(y_test, y_pred)

In [ ]:
print(conf_mat)

In [ ]:
plot_confussion_matrix(y_test, y_pred=y_pred)

In [ ]:
plot_confussion_matrix(y_test, y_pred, normalize=True, target_names=["feature 1", "feature 2"])

### [3.2](#Índice) Métricas de clasificación

Para una descripción de las métricas, podéis mirar en:

* https://scikit-learn.org/stable/modules/model_evaluation.html#model-evaluation

* https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics

In [ ]:
print(classification_report(y_test, y_pred, target_names=["feature 1", "feature 2"]))

In [ ]:
plot_classification_report(y=y_test, y_pred=y_pred, target_names=["feature 1", "feature 2"])

### [3.3](#Índice) Pesos del modelo para cada feature

In [ ]:
plot_feature_importances(logreg)

### [3.4](#Índice) Fronteras de decisión

In [ ]:
plot_decision_boundaries(X_forge, y_forge, logreg.predict(X_forge))

## [4.](#Índice) Clasificación multiclase

Hay algoritmos de clasificación que no permiten la clasificación multiclase. La regresión logística es una excepción. Una técnica común para poder usar un algoritmo que solo permite clasificación binaria a un algoritmo con capacidad de clasificación multiclase es usar la estrategia "Uno contra el Resto" (*One-Vs-Rest*). La estrategia "Uno contra el Resto" no es más que etiquetar la clase que queremos clasificar (por ejemplo con un 1) contra el resto de clases (todas tendrán clase, por ejemplo, 0). De tal forma que tendremos tantos modelos de clasificación binaria como clases queramos clasificar. Para hacer una predicción todos los clasificadores binarios se ejecutan usando un punto de prueba. El clasificador que tiene el *score* más alto en su clase será el "ganador" y esa clase se devuelve como el resultado de la predicción.

Vamos a ver esto con un ejemplo para ver si se entiende mejor:

In [ ]:
X_blob, y_blob = make_blobs(random_state=42, n_features=2)

Vamos a dibujar estos datos a ver cómo se ven:

In [ ]:
fig, ax = plt.subplots()
discrete_scatter(X_blob[:, 0], X_blob[:, 1], y_blob, ax=ax)
ax.set_xlabel("Feature 0")
ax.set_ylabel("Feature 1")
ax.legend(["Class 0", "Class 1", "Class 2"])
_ = plt.title("Dataset con 2 features y 3 target classes")

Entrenamos una regresión logística:

In [ ]:
logreg_multi = LogisticRegression(multi_class="ovr").fit(X_blob, y_blob)

In [ ]:
print("Coef dims: ", logreg_multi.coef_.shape)
print("Intercept dims: ", logreg_multi.intercept_.shape)

In [ ]:
y_pred_blob = logreg_multi.predict(X_blob)
y_pred_blob

### [4.1](#Índice) Fronteras de decisión

Vemos que está usando 3 clases y dos *features* (lo que tenemos en el *dataset*). Vamos a ver las líneas que define cada uno de los tres conjuntos:

In [ ]:
fig, ax = plt.subplots()

discrete_scatter(X_blob[:, 0], X_blob[:, 1], y_blob, ax=ax)
line = np.linspace(-15, 15)
for coef, intercept, color in zip(logreg_multi.coef_, logreg_multi.intercept_, ['b', 'r', 'g']):
    ax.plot(line, -(line * coef[0] + intercept) / coef[1], c=color)
    ax.set_ylim(-10, 15)
    ax.set_xlim(-10, 8)
    ax.set_xlabel("Feature 0")
    ax.set_ylabel("Feature 1")
ax.legend(['Class 0', 'Class 1', 'Class 2', 
           'Line class 0', 'Line class 1','Line class 2'], 
           loc=(1.01, 0.3))

Cada línea muestra la región donde la clase se define como propia o "resto".

Pero ¿qué pasa con la región del medio que ninguna clase define como propia?

Si un punto cae en ese triángulo, ¿a qué clase pertenecerá?

Pues será la clase que tenga el valor más alto, es decir, la clase con la línea más próxima.

In [ ]:
fig, ax = plt.subplots()

plot_2d_classification(logreg_multi, X_blob, fill=True, alpha=.7, ax=ax)
discrete_scatter(X_blob[:, 0], X_blob[:, 1], y_blob, ax=ax)
line = np.linspace(-15, 15)
for coef, intercept, color in zip(logreg_multi.coef_, logreg_multi.intercept_, ['b', 'r', 'g']):
    ax.plot(line, -(line * coef[0] + intercept) / coef[1], c=color)
    ax.legend(['Class 0', 'Class 1', 'Class 2', 
               'Line class 0', 'Line class 1', 'Line class 2'], 
               loc=(1.01, 0.3))
    ax.set_xlabel("Feature 0")
    ax.set_ylabel("Feature 1")

### [4.2](#Índice) Evaluación del modelo

In [ ]:
plot_confussion_matrix(y_pred_blob, y_blob, normalize=False, target_names=['Class 0', 'Class 1', 'Class 2'])

## Otros modelos lineales de clasificación

Le podéis echar un ojo a `LinearSVC` o a `SGDClassifier` u otros en `sklearn.linear_models`. Del primero comentaremos cosas más adelante. Del segundo comentamos cosas usándolo en regresión.

# Ejercicio: Iris dataset

El siguiente ejercicio consiste en analizar como el modelo de regresión logistica es capaz de clasificar el Iris dataset, un dataset de flores que contiene 4 features y 3 especies diferentes de Iris.

Vamos a cargar el dataset y a crear un DataFrame que contenga sus datos.

In [ ]:
iris = load_iris()
iris_df = pd.DataFrame(columns=iris.feature_names, data=iris.data)
iris_df["species"] = iris.target
iris_df["species"] = iris_df["species"].apply(lambda x: iris.target_names[x])
iris_df.columns = [x.replace(" (cm)", "").replace(" ", "_") for x in iris_df.columns]
iris_df.head()

Vamos a visualizar como las diferentes features se relacionan entre ellas para cada una de las clases con un pair plot. En la diagonal se representan los histogramas de como las clases estan distribuidas, mientras que en las otras celdas se representan scatter plots de todos los pares de features.

In [ ]:
from holoviews.operation import gridmatrix

iris_ds = hv.Dataset(iris_df).groupby('species').overlay()
density_grid = gridmatrix(iris_ds, diagonal_type=hv.Distribution, chart_type=hv.Bivariate)
point_grid = gridmatrix(iris_ds, chart_type=hv.Points)

(density_grid * point_grid).opts(
    hv.opts.Bivariate(bandwidth=0.5, cmap=hv.Cycle(values=['Blues', 'Reds', 'Oranges'])),
    hv.opts.Points(size=3, alpha=0.5, tools=["hover"]),
    hv.opts.NdOverlay(batched=False)).opts(width=1000, height=1000)

Para este ejercicio usaremos el dashboard de análisis interactivo para ver como se comporta la regresión logistica con diferentes parámetros.

#### Parte 1: Análisis del modelo por defecto

- Ejecuta el dashboard con los parámetros por defecto.



1. ¿Cuántos errores comete el modelo en el dataset de test?
2. ¿Cuales son las features más importantes para clasificar cada una de las clases?
3. ¿A qué se deben los errores?¿Cómo podrías modificarlo?


In [ ]:
%load ../../solutions/04_01_default.txt

#### Parte 2: Estudio sobre regularización

Selecciona **regularización = "l1"** y **C=0.4**. (Si cambias el toggle de "l1" a "none" podrás comprobar la diferencia sin tener que cambiar C.)

4. ¿A qué crees que se debe el cambio en el plot de fronteras de decisión? ¿Qué ventajas e inconvenientes tiene aplicar regularización en este caso?
5. ¿Hay alguna opción de regularización que ofrezca mejores resultados que "l1"?
6. ¿Modificar el parámetro `fit_intercept` tiene algun efecto sobre la regularización?

In [ ]:
%load ../../solutions/04_02_regularization.txt

**Parte 3: Modificando el peso de las clases**

Finalmente, vamos a comprobar como afecta el parametro class weight al comportamiento de nuestro modelo.

- Añade a class_weight el siguiente valor: `{0 : 0.333, 1 : 0.333, 2 : 0.333}`. Este diccionario representa el valor por el que los errores en cada clase son ponderados, y cuanto mayor sea el peso de una clase, más se penalizará un error en esta. Se aconseja que su suma sea 1; de lo contrario el proceso de regularización se verá afectado.  



7. ¿Cómo afecta el cambiar los pesos de las diferentes clases a los resultados en el dataset de test?

Selecciona regularización "l1" y class_weight = `{0: 0.15, 1: 0.53, 2: 0.33}`.

8. ¿Crees que el modelo resultante es mejor que los anteriores?¿Por qué?

In [ ]:
%load ../../solutions/04_03_weights.txt

In [ ]:
interactive_logistic_regression(iris.data, iris.target,
                                target_names=iris.target_names,
                                feature_names=iris.feature_names)

## [Apéndice](#Índice) Desarrollo matemático

## Otros apuntes

El principal parámetro de los modelos lineales es el parámetro de regularización ($\alpha$ en los modelos de regresión lineales, como la regresión lineal, *Ridge* o Lasso, y `C` en los modelos de clasificación lineales, como la regresión logística, las máquinas de vectores soporte lineales o el clasificador de Gradiente descendiente estocástico. Otra decisión importante en los modelos lineales es el tipo de regularización que queremos usar, principalmente L1 o L2. Dependiendo del problema nos interesará más una estrategia u otra a la hora de seleccionar estos parámetros.

Los modelos lineales son muy rápidos de entrenar y en la predicción. Escalan muy bien a conjuntos de datos muy grandes. Otra fortaleza de los modelos lineales es que suele ser más fácil de ver cómo se obtienen las predicciones. Desafortunadamente no siempre es sencillo saber porqué los coeficientes son como son. Esto es especialmente importante si tus datos tienen dimensiones altamente correlacionadas (multicolinealidad) y será complicado interpretar los coeficientes.

Los modelos lineales, a menudo, se comportan bien cuando el número de dimensiones es largo comparado con el número de datos. También se usan en casos con gran cantidad de datos, principalmente, porque otros modelos no escalan igual de bien y en muchos casos no es posible usarlos. En casos de baja dimensionalidad otros modelos pueden resultar más interesantes puesto que pueden generalizar mejor.

### [A.1](#Índice) Definición del modelo

¿Cómo llegamos a la función logística?

La probabilidad de que un ejemplo $x_i$ sea 1 debido a sus entradas se puede representar de la siguiente forma:

$$ P(y=1|x) $$

Si pensamos que eso se puede representar de forma lineal y tenemos n dimensiones, podríamos escribir algo como:

$ P(y=1|x) = w_0 · x_0 + w_1 · x_1 + ... + w_n · x_n $, donde ($ w_0 = b $ y $ x_0 = 1 $), es decir:

$$ P(y=1|x) = b + w_0 · x_0 + w_1 · x_1 + ... + w_n · x_n $$

Donde $x_i$ representa cada una de las features de nuestro ejemplo, por lo que además de encontrar un peso para cada una de las features, nuestro modelo tendrá un parámetro adicionar que corresponderá a sumar una constante.

Esta constante sumada al output de un modelo lineal se llama intercept.

### [A.2](#Índice) Derivando la funcion sigmoide a partir del ratio de probabilidades

El algoritmo de regresión podría ajustar esos pesos pero, como hemos visto en la gráfica de más arriba eso podría llevar a valores que van de $-\infty$ a $\infty$. Dado que queremos predecir probabilidades, que son valores entre 0. y 1. necesitamos encontrar una función capaz de mapear el output de nuestro modelo linea a ese intervalo.

El ratio de probabilidades (*odds ratio*) es un término que nos puede ayudar. Se define como la probabilidad de que suceda entre la probabilidad de que no suceda. Como nos encontramos en un caso binario, la probabilidad de que suceda es $p$ y la probabilidad de que no suceda es el resto hasta 1, $1-p$. Su ratio se representa así:

$$ odds(p) = \frac{p}{1-p}$$

Si representamos lo anterior para diferentes valores de $p$ tenemos la siguiente gráfica:

In [ ]:
x = np.arange(0, 1, 0.01)
odds = x / (1 - x)

fig, ax = plt.subplots()

ax.plot(x, odds)
ax.set_xlabel("$x$")
ax.set_ylabel(r"$\frac{p}{1-p}$");

Si $p=1$ nuestro ratio se dispara lo cual no es lo que queremos.

Podemos usar el logaritmo natural del ratio y nos da lo siguiente:

In [ ]:
x = np.arange(0.01, 1, 0.01)
odds = np.log(x / (1 - x))

fig, ax = plt.subplots()

ax.plot(x, odds)
ax.set_xlabel("$x$")
ax.set_ylabel(r"$\log(\frac{p}{1-p})$");

Como podemos mapear una combinación lineal de entradas arbitrarias a la función del logaritmo natural de los ratios podemos aprovechar eso para tener:

$$ log\_odds(P(y=1|x)) = w_0 · x_0 + w_1 · x_1 + ... + w_n · x_n $$

($ w_0 = b $ y $ x_0 = 1 $)

Si queremos la $ P(y=1|x) $ podemos buscar la inversa del logaritmo natural ($log\_odds$) para obtenerlo. Si nos aprovechamos de algunas identidades de logaritmos y exponenciales:

$$ y = log(\frac{x}{1-x}) $$

$$ x = log(\frac{y}{1-y}) $$

$$ e^x = \frac{y}{1-y} $$

$$y = (1-y)*e^x $$

$$ y = e^x - y*e^x $$

$$ y + ye^x = e^x $$

$$ y*(1 + e^x) = e^x $$

$$ y = \frac{e^x}{1+e^x} $$

$$ y = \frac{1}{\frac{1}{e^x} + 1} $$

$$ y = \frac{1}{1 + e^{-x}} $$

La última expresión es la inversa de $log\_odds$ y se parece mucho a la función logística o sigmoide (en realidad lo es...):

In [ ]:
x = np.arange(-10, 10, 0.01)
inv_odds = 1 / (1 + np.exp(-x))

fig, ax = plt.subplots()

ax.plot(x, inv_odds)
ax.set_xlabel("$x$")
ax.set_ylabel(r"$\frac{1}{1+e^{-x}}$");

Vaya, ahora todo se restringe a 0 y 1 en el eje *y*.

<div class="alert alert-success">
Explicar que es cada simbolo
</div>

Por tanto, volviendo al principio teníamos que:

$$ log\_odds(P(y=1|x)) = w_0 · x_0 + w_1 · x_1 + ... + w_n · x_n $$

($ w_0 = b $ y $ x_0 = 1 $)

Si simplificamos la notación tenemos:

$$ log\_odds(P(y=1|x)) = w^T·x $$

Si ahora, finalmente, sacamos la inversa de lo anterior nos quedará:

$$ P(y=1|x) = \frac{1}{1+e^{-w^T·x}} $$

## Estimadores de máxima verosimilitud

En inglés se conocen como *maximum likelihood estimators* (MLE). Es un método para obtener los parámetros de un modelo estadístico. El método obtiene los parámetros buscando los valores de los parámetros que maximizan la función de verosimilitud. Las estimaciones se conocen como estimadores de máxima verosimilitud.

Veamos como funciona esto en la práctica. Para ello vamos a usar la distribución normal:

$$ f(x|\mu, \sigma^2) = \frac{1}{\sqrt{2\pi\sigma^2}} exp(\frac{-(x-\mu)^2}{2\sigma^2})  $$

La función de verosimilitud se obtiene de la siguiente forma:

$$ L(\mu, \sigma^2) = f(x_1, x:2,...,x_N|\mu, \sigma^2) = \prod_{i=1}^N f(x_i|\mu, \sigma^2) $$
$$ L(\mu, \sigma^2) = (\frac{1}{2\pi\sigma^2})^{(n/2)} exp(\frac{-\sum_{i=1}^N(x_i-\mu)^2}{2\sigma^2})$$

Lo que queremos hacer es maximizar lo anterior. En la expresión anterior tenemos multiplicaciones pero lo que se suele hacer normalmente es usar el logaritmo de la función de verosimilitud. Y en lugar de maximizar se minimiza usando el negativo del logaritmo de la función de verosimilitud:

$$ ll(\mu, \sigma^2) = \log(L(\mu, \sigma^2)) $$

$$ ll(\mu, \sigma^2) = \log((\frac{1}{2\pi\sigma^2})^{(n/2)} exp(\frac{-\sum_{i=1}^N(x_i-\mu)^2}{2\sigma^2})) $$

$$ nll(\mu, \sigma^2) = -\log(L(\mu, \sigma^2)) $$

$$ nll(\mu, \sigma^2) = -\log((\frac{1}{2\pi\sigma^2})^{(n/2)} exp(\frac{-\sum_{i=1}^N(x_i-\mu)^2}{2\sigma^2})) $$

<div class="alert alert-info">
**[INCISO]** Un breve paréntesis para anotar un par de propiedades de exponenciales y de logaritmos naturales:

$$ e^x · e ^y = e^{(x + y)} $$

$$ ln(x·y) = ln(x) + ln(y) $$

$$ ln(x/y) = ln(x) - ln(y) $$

$$ ln(x^y) = y·ln(x) $$
</div>

Seguimos con MLE...

Si desarrollamos la expresión anterior de $nll$ usando las propiedades de exponenciales y logaritmos nos queda:

$$ nll(\mu, \sigma^2) = -\log((\frac{1}{2\pi\sigma^2})^{(n/2)}) - (\frac{-\sum_{i=1}^N(x_i-\mu)^2}{2\sigma^2}) $$

$$ nll(\mu, \sigma^2) = -\frac{n}{2}\log(\frac{1}{2\pi\sigma^2}) - (\frac{-\sum_{i=1}^N(x_i-\mu)^2}{2\sigma^2}) $$

Para minimizar hemos de usar el gradiente e igualarlo a 0. Si derivamos con respecto a $\mu$:

$$ \frac{\partial{nll(\mu, \sigma^2)}}{\partial{\mu}} = 0 $$

$$ \frac{\partial{nll(\mu, \sigma^2)}}{\partial{\mu}} = \frac{\sum_{i=0}^{N}\mu -\sum_{i=0}^{N}x_i}{2\sigma^2}  = 0 $$

$$ \sum_{i=0}^{N}\mu -\sum_{i=0}^{N}x_i = 0 $$

$$ n·\mu -\sum_{i=0}^{N}x_i = 0 $$

$$ \mu = \frac{\sum_{i=0}^{N}x_i}{n} $$

De la misma forma, derivamos $nll$ con respecto a $\sigma$ e igualamos a 0:

$$ \frac{\partial{nll(\mu, \sigma^2)}}{\partial{\sigma}} = 0 $$

$$ \frac{\partial{nll(\mu, \sigma^2)}}{\partial{\sigma}} = \frac{n}{\sigma} - \frac{\sum_{i=1}^{N}(x_i-\mu)^2}{\sigma^3}= 0 $$

$$ \sigma^2 = \sum_{i=1}^{N}\frac{(x_i-\mu)^2}{n} $$

De esta forma obtendríamos los parámetros de la distribución normal.

Vamos a aplicar lo mismo ahora para el caso de la regresión logística:



La función de verosimilitud será (para una clasificación binaria) para un caso (un ejemplo $(x_i, y_i)$) será:

$$ L(\textbf{w})_i = p(x_i|\textbf{w})^{y_i} · (1-p(x_i|\textbf{w})^{1-y_i}) $$

Si lo generalizamos para los n casos (datos) tenemos la función de verosimilitud:

$$ L(\textbf{w}) = \prod_{i=1}^{N}p(x_i|\textbf{w})^{y_i} · (1-p(x_i|\textbf{w})^{1-y_i}) $$

El *log-likelihood* será:

$$ ll(\textbf{w}) = \sum_{i=1}^{N}({y_i}·\log(p(x_i|\textbf{w})) + {1-y_i}·\log(1-p(x_i|\textbf{w}))) $$

Si toqueteamos lo anterior llegaremos a lo siguiente (saltando pasos):

$$ ll = \sum_{i=1}^{N}y_{i}\textbf{w}^{T}x_{i} - log(1+e^{\textbf{w}^{T}x_{i}}) $$

Si de lo anterior sacamos el gradiente (saltamos pasos):

$$ \bigtriangledown ll = X^{T}(Y - \hat{Y}) $$

El gradiente del *log-likelihood* no es más que la multiplicación de la traspuesta de las entradas por el error de la predicción.

## Referencias

* http://karlrosaen.com/ml/notebooks/logistic-regression-why-sigmoid/

* Sección 4.4.1 de https://web.stanford.edu/~hastie/ElemStatLearn//printings/ESLII_print12.pdf

* https://github.com/martinpella/logistic-reg/blob/master/logistic_reg.ipynb

* https://beckernick.github.io/logistic-regression-from-scratch/

* https://www.datacamp.com/community/tutorials/understanding-logistic-regression-python